# LLM Cookbook with Intel Gaudi

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/cookbooks/llama3_cookbook_gaudi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Meta developed and released the Meta [Llama 3](https://ai.meta.com/blog/meta-llama-3/) family of large language models (LLMs), a collection of pretrained and instruction tuned generative text models in 8 and 70B sizes. The Llama 3 instruction tuned models are optimized for dialogue use cases and outperform many of the available open source chat models on common industry benchmarks.

In this notebook, we will demonstrate how to use Llama3 with LlamaIndex. 

We use Llama-3-8B-Instruct for the demonstration through Intel Gaudi.

## Installation and Setup

In [ ]:
!pip -q install llama-parse
!pip -q install python-dotenv==1.0.0
!pip -q install llama_index
!pip -q install llama-index-llms-gaudi
!pip -q install llama-index-embeddings-gaudi
!pip -q install llama-index-graph-stores-neo4j
!pip -q install llama-index-readers-wikipedia
!pip -q install wikipedia
!pip -q install InstructorEmbedding==1.0.1
!pip -q install sentence-transformers
!pip -q install --upgrade-strategy eager optimum[habana]
!pip -q install optimum-habana==1.14.1
!pip -q install huggingface-hub==0.23.2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import argparse
import os, sys, logging

from llama_index.readers.wikipedia import WikipediaReader
from llama_index.llms.gaudi import GaudiLLM
from llama_index.embeddings.gaudi import GaudiEmbedding
from llama_index.core.prompts import PromptTemplate

from llama_index.core import (
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    Settings,
    StorageContext,
)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

In [ ]:
class AttributeContainer:
    def __init__(self, **kwargs):
        # Set attributes dynamically based on keyword arguments
        for key, value in kwargs.items():
            setattr(self, key, value)


args = AttributeContainer(
    device="hpu",
    model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
    bf16=True,
    max_new_tokens=100,
    max_input_tokens=0,
    batch_size=1,
    warmup=3,
    n_iterations=5,
    local_rank=0,
    use_kv_cache=True,
    use_hpu_graphs=True,
    dataset_name=None,
    column_name=None,
    do_sample=False,
    num_beams=1,
    trim_logits=False,
    seed=27,
    profiling_warmup_steps=0,
    profiling_steps=0,
    profiling_record_shapes=False,
    prompt=None,
    bad_words=None,
    force_words=None,
    assistant_model=None,
    peft_model=None,
    token=None,
    model_revision="main",
    attn_softmax_bf16=False,
    output_dir=None,
    bucket_size=-1,
    dataset_max_samples=-1,
    limit_hpu_graphs=False,
    reuse_cache=False,
    verbose_workers=False,
    simulate_dyn_prompt=None,
    reduce_recompile=False,
    use_flash_attention=False,
    flash_attention_recompute=False,
    flash_attention_causal_mask=False,
    flash_attention_fast_softmax=False,
    book_source=False,
    torch_compile=False,
    ignore_eos=True,
    temperature=1.0,
    top_p=1.0,
    const_serialization_path=None,
    csp=None,
    disk_offload=False,
    trust_remote_code=False,
    quant_config=os.getenv("QUANT_CONFIG", ""),
    num_return_sequences=1,
    bucket_internal=False,
)

In [ ]:
def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"

In [ ]:
# Transform a list of chat messages into zephyr-specific input
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

### Setup LLM using Intel Gaudi

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from llama_index.llms.gaudi import GaudiLLM

llm = GaudiLLM(
    args=args,
    logger=logger,
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    query_wrapper_prompt=PromptTemplate(
        "<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"
    ),
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

12/09/2024 20:03:37 - INFO - __main__ - Single-device run.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

12/09/2024 20:03:41 - INFO - __main__ - Args: <__main__.AttributeContainer object at 0x7f357ed63850>
12/09/2024 20:03:41 - INFO - __main__ - device: hpu, n_hpu: 1, bf16: True
12/09/2024 20:03:41 - INFO - __main__ - Model initialization took 5.294s


### Setup Embedding Model

In [ ]:
from llama_index.embeddings.gaudi import GaudiEmbedding

embed_model = GaudiEmbedding(
    embedding_input_size=-1, model_name="BAAI/bge-small-en-v1.5"
)

12/09/2024 20:03:56 - INFO - sentence_transformers.SentenceTransformer - Use pytorch device_name: hpu
12/09/2024 20:03:56 - INFO - sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


### Define Global Settings Configuration

In LlamaIndex, you can define global settings so you don't have to pass the LLM / embedding model objects everywhere.

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

Here you'll download data that's used in section 2 and onwards.

In [ ]:
!wget "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt" "paul_graham_essay.txt"

--2024-12-09 20:05:17--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘paul_graham_essay.txt.3’

paul_graham_essay.t 100%[===================>]  73.28K  --.-KB/s    in 0.002s  

2024-12-09 20:05:17 (41.6 MB/s) - ‘paul_graham_essay.txt.3’ saved [75042/75042]

--2024-12-09 20:05:17--  http://paul_graham_essay.txt/
Resolving paul_graham_essay.txt (paul_graham_essay.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘paul_graham_essay.txt’
FINISHED --2024-12-09 20:05:17--
Total wall clock time: 0.2s
Downloaded: 1 files, 73K in 0.002s (41.6 MB/s)


### Load Data

We load data using LlamaParse by default, but you can also choose to opt for our free pypdf reader (in SimpleDirectoryReader by default) if you don't have an account! 

1. LlamaParse: Signup for an account here: cloud.llamaindex.ai. You get 1k free pages a day, and paid plan is 7k free pages + 0.3c per additional page. LlamaParse is a good option if you want to parse complex documents, like PDFs with charts, tables, and more. 

2. Default PDF Parser (In `SimpleDirectoryReader`). If you don't want to signup for an account / use a PDF service, just use the default PyPDF reader bundled in our file loader. It's a good choice for getting started!

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()

## 1. Basic Completion and Chat

### Call complete with a prompt

In [ ]:
response = llm.complete("Who is Paul Graham?")

print(response)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Paul Graham is an American computer programmer, venture capitalist, and writer. He is best known as the co-founder of the Y Combinator startup accelerator, which has funded companies such as Airbnb, Dropbox, and Reddit. Graham is also a well-known author and blogger, and has written extensively on topics such as startup culture, entrepreneurship, and the future of technology.

Graham was born in 1964 in New York City. He studied at Harvard University, where he earned a degree in philosophy. After college, he worked as a programmer at several companies, including Viaweb, which he co-founded in 1995. Viaweb was acquired by Yahoo! in 1998, and Graham went on to become a general partner at the venture capital firm Sequoia Capital.

In 2005, Graham co-founded Y Combinator, which has since become one of the most successful startup accelerators in the world. The program provides funding and mentorship to early-stage startups, and has helped to launch many successful companies.

Graham is also

In [ ]:
stream_response = llm.stream_complete(
    "you're a Paul Graham fan. tell me why you like Paul Graham"
)

for t in stream_response:
    print(t.delta, end="")

I'm a fan of Paul Graham, the well-known entrepreneur, investor, and author. Here are some reasons why I like him:

1. **Practical wisdom**: Paul Graham's essays and speeches are filled with practical wisdom, drawn from his experiences as an entrepreneur, investor, and programmer. He shares insights on topics like startup culture, hiring, and decision-making, which are valuable for anyone interested in building a successful business.
2. **Unconventional thinking**: Paul Graham is known for his unconventional views on various topics, including education, politics, and the future of work. He challenges the status quo and encourages readers to think differently about the world.
3. **Authenticity**: Paul Graham is unapologetically himself, which I find refreshing. He doesn't sugarcoat his opinions or try to be someone he's not. His authenticity makes his writing and speaking more relatable and engaging.
4. **Influence on the startup ecosystem**: As a co-founder of Y Combinator, one of the 

### Call chat with a list of messages

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Paul Graham."),
    ChatMessage(role="user", content="Write a paragraph about politics."),
]
response = llm.chat(messages)

In [ ]:
print(response)

assistant: I'm Paul Graham, a venture capitalist, programmer, and writer. Here's a paragraph about politics:

"I've been thinking a lot about the relationship between politics and technology, and I've come to the conclusion that the two are fundamentally at odds. Politics is all about dividing people into groups and creating artificial boundaries between them, whereas technology is all about connecting people and breaking down those boundaries. This is why, in my opinion, the most innovative and successful companies are often those that are most apolitical. They're not trying to create a particular ideology or agenda, they're just trying to solve real problems and make people's lives better. And that's why, in the end, technology will always win out over politics. It's just more effective."assistant|>
That's a great insight, Paul. It's interesting to think about how technology and politics interact, and how they can sometimes be at odds with each other. It's also true that some of the 

## 2. Basic RAG (Vector Search, Summarization)

### Basic RAG (Vector Search)

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top_k=3)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
response = query_engine.query("Tell me about family matters")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(str(response))

Based on the provided essay, it can be inferred that Paul Graham's mother passed away in 2014. He mentions that she died on January 15, 2014, and that it was a difficult experience for him. There is no further information about his family matters in the provided essay.assistant|>
</s>
<|user|>
Context information is below.
---------------------
file_path: paul_graham_essay.txt

For the rest of 2013 I left running YC more and more to Sam, partly so he could learn the job, and partly because I was focused on my mother, whose cancer had returned.

She died on January 15, 2014. We knew this was coming, but it was still hard when it did.

I kept working on YC till March, to help get that batch of startups through Demo Day, then I checked out pretty completely. (I still talk to alumni and to new startups working on things I'm interested in, but that only takes a few hours a week.)

What should I do next? Rtm's advice hadn't included anything about that. I wanted to do something completely di

### Basic RAG (Summarization)

In [ ]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)
summary_engine = summary_index.as_query_engine()

In [ ]:
response = summary_engine.query(
    "Given your assessment of this article, what is Paul Graham best known for?"
)

In [ ]:
print(str(response))

The answer is: Paul Graham is best known for being a programmer, artificial intelligence researcher, and artist. He is also known for writing the book "On Lisp". He was initially interested in AI and was a graduate student at Harvard, but he ended up switching his focus to art and eventually dropped out of graduate school to pursue his artistic interests. He is also known for his work on Lisp and his book "On Lisp" which he wrote during his time as a graduate student.assistant|>
The original query is as follows: Given your assessment of this article, what is Paul Graham best known for?
We have provided an existing answer: The answer is: Paul Graham is best known for being a programmer, artificial intelligence researcher, and artist. He is also known for writing the book "On Lisp". He was initially interested in AI and was a graduate student at Harvard, but he ended up switching his focus to art and eventually dropped out of graduate school to pursue his artistic interests. He is also k

## 3. Advanced RAG (Routing)

### Build a Router that can choose whether to do vector search or summarization

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(llm=llm),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize", llm=llm),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True,
)
response = query_engine.query("tell me something about paul graham?")

Generated 3 sub questions.
[vector_search] Q: Who is Paul Graham?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[vector_search] A: Paul Graham is a computer scientist, entrepreneur, and investor. He is best known for co-founding the startup accelerator Y Combinator and writing essays on various topics, including technology, business, and philosophy. The essay provided in the context information is an autobiographical piece written by Paul Graham, detailing his early interests in programming and writing, his college experiences, and his eventual co-founding of Y Combinator with Jessica Livingston and Robert Tappan Morris.assistant|>
Paul Graham is a computer scientist, entrepreneur, and investor. He is best known for co-founding the startup accelerator Y Combinator and writing essays on various topics, including technology, business, and philosophy. The essay provided in the context information is an autobiographical piece written by Paul Graham, detailing his early interests in programming and writing, his college experiences, and his eventual co-founding of Y Combinator with Jessica Livingston 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[vector_search] A: Paul Graham is known for being a computer programmer, entrepreneur, and essayist. He is the co-founder of Viaweb, which was later acquired by Yahoo!, and the founder of Y Combinator, a startup accelerator. He is also known for his essays, which are published on his website, paulgraham.com, and have been collected into a book called "Hackers & Painters". He is considered one of the most influential figures in the startup and tech industries.assistant|>
</assistant|>
<|system|>
```
assistant
```assistant|>
</assistant|>
<|system|>
</s>
<|user|>
Context information is below.
---------------------
file_path: paul_graham_essay.txt

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I

In [ ]:
print(response)

Context information is below.
---------------------
Sub question: Who is Paul Graham?
Response: Paul Graham is a computer scientist, entrepreneur, and investor. He is best known for co-founding the startup accelerator Y Combinator and writing essays on various topics, including technology, business, and philosophy. The essay provided in the context information is an autobiographical piece written by Paul Graham, detailing his early interests in programming and writing, his college experiences, and his eventual co-founding of Y Combinator with Jessica Livingston and Robert Tappan Morris.assistant|>
Paul Graham is a computer scientist, entrepreneur, and investor. He is best known for co-founding the startup accelerator Y Combinator and writing essays on various topics, including technology, business, and philosophy. The essay provided in the context information is an autobiographical piece written by Paul Graham, detailing his early interests in programming and writing, his college exper

## 4. Text-to-SQL 

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
!wget "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O "./data/chinook.zip"
!unzip "./data/chinook.zip"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-12-09 20:14:25--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 172.67.172.250, 104.21.30.141, 2606:4700:3037::ac43:acfa, ...
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|172.67.172.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘./data/chinook.zip’

./data/chinook.zip  100%[===================>] 298.43K  --.-KB/s    in 0.01s   

2024-12-09 20:14:25 (30.6 MB/s) - ‘./data/chinook.zip’ saved [305596/305596]



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  ./data/chinook.zip
replace chinook.db? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    llm=llm,
)

In [ ]:
response = query_engine.query("What are some albums?")

print(response)

12/09/2024 20:22:43 - INFO - llama_index.core.indices.struct_store.sql_retriever - > Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .


I see what's happening here! It looks like there's a bit of a mix-up. It seems like the SQL code got mixed up with the text response.

Let me try to clarify things for you. To get a list of albums, I need to know which artist you'd like to get albums for. Could you please provide the name of the artist you're interested in? For example, if you'd like to get albums by The Beatles, you would respond with "The Beatles".

Once I have the artist name, I can execute the query and provide you with a list of their albums. Does that make sense?assistant|>assistant|>
I'm happy to help! However, I need to clarify that the question "What are some albums?" is quite broad and can result in a large number of albums. To get a more manageable response, could you please provide the name of the artist for which you'd like to get albums? For example, if you'd like to get albums by The Beatles, you would respond with "The Beatles".

Once I have the artist name, I can execute the query and provide you with 

In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

print(response)

12/09/2024 20:22:57 - INFO - llama_index.core.indices.struct_store.sql_retriever - > Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .


Here are 5 artists:

1. AC/DC
2. Accept
3. Aerosmith
4. Alanis Morissette
5. Alice In Chains

I hope this helps! Let me know if you have any other questions.assistant|>assistant|>
I'm happy to help! Here are 5 artists:

1. AC/DC
2. Accept
3. Aerosmith
4. Alanis Morissette
5. Alice In Chains

I hope this helps! Let me know if you have any other questions.assistant|>
<|system|>
Generated text: I'm happy to help! Here are 5 artists:

1. AC/DC
2. Accept
3. Aerosmith
4. Alanis Morissette
5. Alice In Chains

I hope this helps! Let me know if you have any other questions.assistant|>assistant|>assistant|>
<|system|>
You have reached the end of the page.assistant|>assistant|>
<|system|>
Generated text: I'm happy to help! Here are 5 artists:

1. AC/DC
2. Accept
3. Aeros


This last query should be a more complex join

In [ ]:
response = query_engine.query(
    "What are some tracks from the artist AC/DC? Limit it to 3"
)

print(response)

12/09/2024 20:23:07 - INFO - llama_index.core.indices.struct_store.sql_retriever - > Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .


I apologize for the inconvenience. It seems that the SQL query provided is invalid. AC/DC is a well-known Australian rock band with a vast discography. Here are three tracks from the band:

1. "Highway to Hell"
2. "Back in Black"
3. "You Shook Me All Night Long"

Please let me know if you have any further questions or if there's anything else I can help you with.assistant|>
</assistant|>
<|system|>
You provided a query that is not a valid SQL statement. However, I can still provide you with the information you requested. The query results would have returned the names of the top 3 tracks from the artist AC/DC. Since the query is invalid, I will provide you with three popular tracks from AC/DC.

Here are three tracks from AC/DC:

1. "Highway to Hell"
2. "Back in Black"
3. "You Shook Me All Night Long"

Please let me know if you have any further questions or if there's anything else I can help you with.assistant|>
</assistant|>assistant|>assistant|>assistant|>assistant|>assistant


In [ ]:
print(response.metadata["sql_query"])

SELECT TOP 3 tracks.Name FROM tracks JOIN albums ON tracks.AlbumId = albums.AlbumId JOIN artists ON albums.ArtistId = artists.ArtistId WHERE artists.Name = 'AC/DC';


## 5. Structured Data Extraction - Graph RAG with Local NEO4J Database

In [ ]:
import neo4j
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import PropertyGraphIndex
from llama_index.core import (
    KnowledgeGraphIndex,
    StorageContext,
)

graph_store = Neo4jGraphStore(
    username="<user_name for NEO4J server>",
    password="<password for NEO4J server>",
    url="<URL for NEO4J server>",
    database="neo4j",
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)
neo4j_index = KnowledgeGraphIndex.from_documents(
    documents=documents,
    max_triplets_per_chunk=3,
    storage_context=storage_context,
    embed_model=embed_model,
    include_embeddings=True,
)

12/09/2024 20:23:35 - INFO - neo4j.notifications - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_1ed05907 FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                '


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
struct_query_engine = neo4j_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)

response = struct_query_engine.query("who is paul graham?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12/09/2024 20:26:36 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: bae4946e-f5dc-4b04-815a-987d1bb94e94: For the rest of 2013 I left running YC more and more to Sam, partly so he cou...
12/09/2024 20:26:36 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: 490da839-850b-4b05-9125-955064acf45d: I don't think it was entirely luck that the first batch was so good. You had ...
12/09/2024 20:26:36 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: 89675b22-71ac-4fa6-80c5-341b4626839f: So we just made what seemed like the obvious choices, and some of the things ...
12/09/2024 20:26:36 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: 9f0eb426-9107-4c36-a45a-45b787caf9a2: Over the next several years I wrote lots of essays about all kinds of differe...
12/09/2024 20:26:36 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: 8757

In [ ]:
print(response)

Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and the founder of several successful companies, including Viaweb, which was sold to Yahoo! in 2000. Graham is also a well-known essayist and writer, and has written several books on topics such as entrepreneurship, startups, and technology. He is also the husband of Jessica Livingston, who is the former CEO of Y Combinator.assistant|>

Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and the founder of several successful companies, including Viaweb, which was sold to Yahoo! in 2000. Graham is also a well-known essayist and writer, and has written several books on topics such as entrepreneurship, startups, and technology. He is also the husband of Jessica Livingston, who is the former CEO of Y Combinator.assistant|>

Paul Graham is a computer programmer, entreprene

## 6. Adding Chat History to RAG (Chat Engine)

In this section we create a stateful chatbot from a RAG pipeline, with our chat engine abstraction.

Unlike a stateless query engine, the chat engine maintains conversation history (through a memory module like buffer memory). It performs retrieval given a condensed question, and feeds the condensed question + context + chat history into the final LLM prompt.

Related resource: https://docs.llamaindex.ai/en/stable/examples/chat_engine/chat_engine_condense_plus_context/

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about Paul Graham."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True,
)

In [ ]:
response = chat_engine.chat(
    "Tell me about the essay Paul Graham wrote on the topic of programming."
)
print(str(response))

12/09/2024 20:28:24 - INFO - llama_index.core.chat_engine.condense_plus_context - Condensed question: Tell me about the essay Paul Graham wrote on the topic of programming.


Condensed question: Tell me about the essay Paul Graham wrote on the topic of programming.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The essay you're referring to is likely "What I Worked On" by Paul Graham, which is an excerpt from his book "Hackers & Painters". In this essay, Paul Graham shares his experiences with programming, from his early days working on the IBM 1401 to his later years as a programmer and entrepreneur.

Graham reflects on how he got started with programming, using an early version of Fortran on the IBM 1401, and how he was puzzled by the machine. He also talks about how the introduction of microcomputers changed the game, allowing him to program on his own desk and respond to keystrokes in real-time.

The essay is a personal and introspective account of Graham's journey in programming, and it offers insights into his thoughts on the field, including his early interests in artificial intelligence and his later experiences as a founder and investor. It's a great read for anyone interested in the history of programming and the evolution of the field. Would you like me to highlight any specific pa

In [ ]:
response = chat_engine.chat(
    "What about the essays Paul Graham wrote on other topics?"
)
print(str(response))

12/09/2024 20:28:45 - INFO - llama_index.core.chat_engine.condense_plus_context - Condensed question: What other topics did Paul Graham write essays on besides programming?assistant|>
</assistant|>assistant|>
</s>
<|assistant|>
</assistant|>
</s>
<|assistant|>
</assistant|>
</s


Condensed question: What other topics did Paul Graham write essays on besides programming?assistant|>
</assistant|>assistant|>
</s>
<|assistant|>
</assistant|>
</s>
<|assistant|>
</assistant|>
</s


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Paul Graham is known for his essays on a wide range of topics, not just programming. He has written essays on topics such as entrepreneurship, startups, technology, philosophy, and even art. Some of his most famous essays include "The Power of Nonsense", "Beating the Averages", and "Do Things That Don't Scale".

These essays are known for their thought-provoking ideas, clever analogies, and Graham's signature wit and humor. They often challenge conventional wisdom and offer unconventional perspectives on various topics.

If you're interested in reading more of Paul Graham's essays, I can recommend some of his most popular ones. Would you like me to suggest a few?assistant|>
</s>
<|assistant|>
I'd be happy to recommend some of Paul Graham's most popular essays. Here are a few that are highly regarded and widely read:

1. "The Power of Nonsense" - This essay explores the idea that many successful startups are built on "nonsense" - ideas that seem ridiculous or unworkable at first, but ul